<a href="https://colab.research.google.com/github/kuuumaaaa/data_analyze_technique_in_kaggle/blob/main/ch01/ch01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

学習データとテストデータの読み込み






In [2]:
train = pd.read_csv('/content/drive/MyDrive/参考書/Kaggleで勝つデータ分析の技術/kagglebook-master/input/ch01-titanic/train.csv')
train_x = train.drop(['Survived'],axis =1)
train_y = train['Survived']
test = pd.read_csv('/content/drive/MyDrive/参考書/Kaggleで勝つデータ分析の技術/kagglebook-master/input/ch01-titanic/test.csv')
test_x = test.copy()

# EDAのポイント
カラムの把握、各カラムの方や値の分布、欠損値、外れ値、目的変数と各変数の相関や関係性
- 変数の平均・標準偏差・最大・最小・分位点
- カテゴリ変数の値の種類数
- 変数の欠損値の値
- 変数間の相関関係

バリデーションデータへの分け方

In [3]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

#特徴量の作成
今回GBDTのモデルで作成
- IDは予測に寄与しないため削除
- Name,Ticket,Cabinについて煩雑な処理が必要なため削除
- 文字はlabel encoding
- 欠損はそのまま

In [4]:
from  sklearn.preprocessing import LabelEncoder

# 変数PassengerIdを除去
train_x = train_x.drop(['PassengerId'],axis=1)
test_x = test_x.drop(['PassengerId'],axis=1)

# 変数Name,Ticket,Cabiを削除
train_x = train_x.drop(['Name','Ticket','Cabin'],axis=1)
test_x = test_x.drop(['Name','Ticket','Cabin'],axis=1)

# それぞれのカテゴリ変数にlabel encoding
for c in ['Sex','Embarked']:
  # 学習データに基づいてどう変換するかを定める
  le = LabelEncoder()
  le.fit(train_x[c].fillna('NA'))

  # 学習データ、テストデータを変換する
  train_x[c] = le.transform(train_x[c].fillna('NA'))
  test_x[c] = le.transform(test_x[c].fillna('NA'))

#モデルの作成
- GBDTのライブラリの一つであるxgboostを用いてモデルを学習

In [5]:
from xgboost import XGBClassifier

# モデルの作成及び学習データを与えての学習
model = XGBClassifier(n_estimators=20, random_state=71)
# n_estimators : Number of gradient boosted trees. Equivalent to number of boosting rounds.(決定木の数)
# random_state (Optional[Union[numpy.random.RandomState, int]]) –Random number seed.
model.fit(train_x,train_y)

# テストデータの予測値を確率で出力
pred = model.predict_proba(test_x)[:,1]
# Predict the probability of each X example being of a given class.

# テストデータの予測値を二値に変換する
pred_label = np.where(pred > 0.5,1,0)
# 0.5以上を1に、それ以下を0にする

# 提出用ファイルの作成
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':pred_label})
submission.to_csv('/content/drive/MyDrive/参考書/Kaggleで勝つデータ分析の技術/outputs/ch01/submission_first.csv',index=False)

#モデルの評価
- バリデーションデータへの予測精度をスコアで表すことで評価
- 今回はクロスバリデーションを使用し評価
- バリデーションとpublic leaderboardには原因がない限り近い値になる

In [6]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

# 各foldのスコアを保存するリスト
scores_accuracy = []
scores_logloss = []

# クロスバリデーションを行う
# 学習データを４つに分割し、うち一つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
  # 学習データを学習データとバリデーションデータに分ける
  tr_x,va_x = train_x.iloc[tr_idx],train_x.iloc[va_idx]
  tr_y,va_y = train_y.iloc[tr_idx],train_y.iloc[va_idx]

  # モデルの学習を行う
  model = XGBClassifier(n_estimators=20, random_state=71)
  model.fit(tr_x,tr_y)

  # バリデーションデータの予測値を確率で出力
  va_pred = model.predict_proba(va_x)[:,1]

  # バリデーションデータでのスコアを計算する
  logloss = log_loss(va_y,va_pred)
  accuracy = accuracy_score(va_y,va_pred> 0.5)

  # そのfoldのスコアを保存する
  scores_logloss.append(logloss)
  scores_accuracy.append(accuracy)

  # 各foldのスコアの平均を出力する
  logloss = np.mean(scores_logloss)
  accuracy = np.mean(scores_accuracy)
  print(f'logloss:{logloss:.4f},accuracy: {accuracy:4f}')

logloss:0.3972,accuracy: 0.834081
logloss:0.4155,accuracy: 0.820628
logloss:0.4265,accuracy: 0.814649
logloss:0.4270,accuracy: 0.814815


#モデルのチューニング
ハイパーパラメータのチューニング、今回はグリッドサーチを用いる。

In [19]:
import itertools

# チューニング候補とするパラメータを準備
param_space = {
    'max_depth' : [3,5,7],
    'min_child_weight' : [1.0, 2.0, 4.0]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

# 各パラメータの組み合わせ、それに対するスコアを帆s損するリスト
params = []
scores = []

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth, min_child_weight in param_combinations:

  score_folds = []
  # クロスバリデーションを行う
  # 学習データを４つに分割し、うち一つをバリデーションデータとすることをバリデーションデータを変えて繰り返す。
  kf = KFold(n_splits=4, shuffle=True, random_state=123456)
  for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = XGBClassifier(n_estimators=20, random_state=71,
                          max_depth=max_depth, min_child_weight=min_child_weight)
    model.fit(tr_x, tr_y)

    # バリデーションデータでのスコアを計算し、保存する
    va_pred = model.predict_proba(va_x)[:,1]
    logloss = log_loss(va_y,va_pred)
    score_folds.append(logloss)
    
    # 各 foldのスコアを平均する
    score_mean = np.mean(score_folds)

    # パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'maxdepth: {best_param[0]}, min_child_weight: {best_param[1]}')

maxdepth: 7, min_child_weight: 2.0


 #アンサンブル
 

In [22]:
from sklearn.linear_model import LogisticRegression

# xgboostモデル
model_xgb  = XGBClassifier(n_estimators=20, random_state=71)
model_xgb.fit(train_x, train_y)
pred_xgb = model_xgb.predict_proba(test_x)[:,1]

# ロジスティック回帰モデル
# xgboostモデルとは異なる特徴量を入れる必要があるので、別途train_x2,test_x2を作成
model_lr = LogisticRegression(solver='lbfgs' , max_iter=300)
model_lr.fit(train_x2,train_y)
pred_lr = model_lr.predict_proba(test_x2)[:,1]

# 予測値の加重平均をとる
pred - pred_xgb * 0.8 + pred_lr * 0.2
pred_label = np.where(pred > 0.5 ,1,0)

NameError: ignored